In [1]:
import pandas as pd
import numpy as np

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('retina', 'png')

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.patches import Rectangle, Circle, Ellipse, Patch
from matplotlib.lines import Line2D

from ete3 import Tree

import seaborn as sns
sns.set() # sets seaborn default "prettyness:
sns.set_style("white")
sns.set_context("notebook")

ModuleNotFoundError: No module named 'ete3'

Code for plotting the tree:

In [ ]:
def make_comb_tree(t):
    for node in t.traverse():
        if not node.is_leaf():
            node.children = sorted(node.children, key=lambda c: len(c.get_leaves()), reverse=False)

def plot_tree(t, ax, leaf_colors=None, show_inner_nodes=False, flip=False, fontsize=10, 
              text_offset=None, axis=True, linewidth=0.8, markers=True, 
              labels=True, markersize=3, margins=(0.5, 1, 0.5, 1)): # top, right, bottom, left

    y_offset = len(t.get_leaves())
    for node in t.traverse("preorder"):
        node.x_offset = node.dist + sum(x.dist for x in node.get_ancestors())
        if node.is_leaf():
            y_offset -= 1
            node.y_offset = y_offset

    for node in t.traverse("postorder"):
        if not node.is_leaf():
            node.y_offset = sum(x.y_offset for x in node.children) / len(node.children)

    horizontal_lines = list()
    vertical_lines = list()
    node_coords = list()
    leaf_coords = list()
    max_x_offset = 0
    for node in t.traverse("postorder"):
        max_x_offset = max(max_x_offset, node.x_offset)
        node_coords.append((node.x_offset, node.y_offset))
        if node.is_leaf():
            leaf_coords.append([node.name, node.x_offset, node.y_offset])
        if not node.is_root():
            y = node.y_offset
            horizontal_lines.append(([node.up.x_offset, node.x_offset], [y, y]))
        if not node.is_leaf():
            c = sorted(node.children, key=lambda x: x.y_offset)
            bottom, top = c[0], c[-1]
            x = node.x_offset
            vertical_lines.append(([x, x],[bottom.y_offset, top.y_offset]))

    
    # shift the tree to put leaves at zero
    for i in range(len(horizontal_lines)):
        horizontal_lines[i][0][0] -= max_x_offset
        horizontal_lines[i][0][1] -= max_x_offset
    for i in range(len(vertical_lines)):
        vertical_lines[i][0][0] -= max_x_offset
        vertical_lines[i][0][1] -= max_x_offset
    for i in range(len(leaf_coords)):
        leaf_coords[i][1] -= max_x_offset

    if flip:
        l = len(leaf_coords)
        horizontal_lines = [([l-y1-1, l-y2-1], [-x1, -x2]) for ((x1, x2),(y1, y2)) in horizontal_lines]
        vertical_lines = [([l-y1-1, l-y2-1], [-x1, -x2]) for ((x1, x2),(y1, y2)) in vertical_lines]
        leaf_coords = [[n, x, y] for (n, y, x) in leaf_coords]
        
    # draw the tree:
    for x in horizontal_lines:
        ax.plot(*x, c='black', linewidth=linewidth)
    for x in vertical_lines:
        ax.plot(*x, c='black', linewidth=linewidth)

#     for tup in node_coords:
#         ax.plot(*tup, c='black', marker="o")

    if text_offset is None:
        text_offset = max_x_offset / 20

    for name, x, y in leaf_coords:
        if flip:
            verticalalignment='top'
            horizontalalignment='center'
            rotation='vertical'
        else:
            verticalalignment='center'
            horizontalalignment='left'
            rotation='horizontal'
        if labels:
            ax.text(x+text_offset, y, name, fontsize=fontsize,
                    verticalalignment=verticalalignment, 
                    horizontalalignment=horizontalalignment,
                    rotation=rotation)
        if markers:
            if leaf_colors is None:
                color = 'black'
            else:
                color = leaf_colors[name]
            ax.plot(x, y, c=color, marker="o", ms=markersize)

    if flip:
        ax.set_ylim(-margins[1], -(margins[3]-max_x_offset))
        ax.set_xlim(-margins[2], len(leaf_coords)-1+margins[0])
    else:
        ax.set_xlim(-margins[3]-max_x_offset, margins[1])
        ax.set_ylim(-margins[2], len(leaf_coords)-1+margins[0])

    if flip:
        ax.get_xaxis().set_visible(False)
    else:
        ax.get_yaxis().set_visible(False)

    ax.spines['top'].set_visible(False) 
    ax.spines['right'].set_visible(False)
    if flip:
        ax.yaxis.set_major_locator(plt.MaxNLocator(4))
        ax.spines['bottom'].set_visible(False) 
    else:
        ax.spines['left'].set_visible(False) 
        ax.xaxis.set_major_locator(plt.MaxNLocator(4))
        
    if not axis:
        ax.axis('off')

    
    return leaf_coords            

Read the copying matrix:

In [ ]:
# coancestry_matrix = pd.read_csv('Chromopainter/all_autosomes/all_autosomes_mapstate.csv').drop(columns=['relXmat'])
# coincidence_matrix = pd.read_csv('Chromopainter/all_autosomes/all_autosomes_meancoincidence.csv').drop(columns=['relXmat'])
matrix = pd.read_csv('/home/eriks/baboondiversity/people/eriks/baboon_first_analysis/steps/fs/all_autosomes_linked.regionchunkcounts.out',
                    sep=' ', header=0, index_col=0).drop(columns=['num.regions'])
matrix

Read tree from xml (requires lxml package via conda):

In [ ]:
df = pd.read_xml('/home/eriks/baboondiversity/people/eriks/baboon_first_analysis/steps/fs/all_autosomes_linked_tree.xml')
tree =  df.Tree[3]

Get order of individuals in tree:

In [ ]:
matrix_order = [node.name for node in Tree(tree).traverse("postorder") if node.is_leaf()]

Order matrix accordingly (and reverse x axis to make the plot like Eriks)

In [ ]:
ordered_matrix = matrix.loc[matrix_order[::-1], matrix_order]

Somehow get the species color for each individual (these are just dummies):

In [ ]:
#df.Pop[2][1:-1].split(')(')

In [ ]:
species_colors = dict(zip(matrix_order, 
                       np.random.choice(['#e31a1c', '#FFB100','#33a02c', '#1f78b4'], 
                                        size=len(matrix_order))))
cluster_colors = dict(zip(matrix_order, 
                       np.random.choice(['#e31a1c', '#FFB100','#33a02c', '#1f78b4'], 
                                        size=len(matrix_order))))

In [ ]:
idfile_path = "/home/eriks/baboondiversity/data/PG_panu3_phased_chromosomes_4_7_2021/idfile_8_cluster.ids"
idfile = pd.read_csv(idfile_path, sep=" ", names=["PGDP_ID", "pop", "inclusion"])
idfile["ID_index"] = idfile.index

meta_data_samples = pd.read_csv("../data/Papio_metadata_with_clustering.txt", sep =" ")
mapping = {}
for i, row in meta_data_samples.iterrows():
    if row.PGDP_ID[0] != "P":
        mapping["Sci_"+row.PGDP_ID] = row.Origin
    else:
        mapping[row.PGDP_ID] = row.Origin
meta_data_samples["PGDP_ID"] = mapping.keys()
mycols = sns.color_palette(["#BEE39C", "#1F681F", "#FFE7AF",
                            "#FFD062", "#9EADB2", "#258CC1", "#EA3324", "#C06D34"])
name_order = sorted(meta_data_samples.loc[meta_data_samples.PGDP_ID.isin(matrix_order)]["C_origin"].unique())
color_dir = {}
for i in range(len(name_order)):
    color_dir[name_order[i]] = mycols[i]
# This order of color is based on the alphabetical ordering of the 14 clusters
mycols_cmap = ListedColormap(mycols.as_hex())

mycols

In [ ]:
color_df = pd.DataFrame()
color_df["Color"] = meta_data_samples.loc[meta_data_samples.PGDP_ID.isin(matrix_order)].reset_index().C_origin.map(color_dir)
color_df["PGDP_ID"] = meta_data_samples.loc[meta_data_samples.PGDP_ID.isin(matrix_order)].reset_index().PGDP_ID
species_color = pd.Series(color_df["Color"].values,index=color_df["PGDP_ID"]).to_dict()

In [ ]:
mycols = sns.color_palette(["#BEE39C", "#88ad00", "#3BAC3B", "#68E068", "#77c458", "#1F681F", "#FFE7AF",
                            "#FFD062", "#FFDE90", "#FFBD00", "#9EADB2", "#258CC1", "#EA3324", "#C06D34"])
name_order = idfile.loc[idfile.inclusion==1].sort_values(by="population").population.unique()
color_dir = {}
for i in range(len(name_order)):
    color_dir[name_order[i]] = mycols[i]
# This order of color is based on the alphabetical ordering of the 14 clusters
mycols_cmap = ListedColormap(mycols.as_hex())
mycols

Plot the figure:

In [ ]:
# three panels
fig, ((axA, ax1), (axB, ax2), (axC, ax3), (axD, ax4))  = plt.subplots(4, 2, sharex='col', sharey='row',
                        gridspec_kw={'height_ratios':[10,1,1,40], 
                                     'width_ratios':[1,40],
                                     'hspace':0.01,
                                     'wspace':0.01}, 
                        figsize=(8, 9))

# remove axes we do not need
axA.remove()
axB.remove()
axC.remove()

# plot the tree in top panel
leaf_info = plot_tree(Tree(tree), ax1, flip=True, 
                      axis=False, linewidth=0.6,
                      markers=True, 
                      markersize=1,
                      labels=False,
                      margins=(0, 0, 0, 0), 
                      fontsize=4)

# add colored rectangles for or each individual cluster membership
for i, indiv in enumerate(matrix_order):
    ax2.add_patch(Rectangle((i, 0), 1, 1, linewidth=0, edgecolor='black', facecolor=species_color[indiv]))
ax2.text(-1, 0.5, '8 clusters:', fontsize=10, verticalalignment='center', horizontalalignment='right')  
    
# add colored rectangles for each individual to the middle
for i, indiv in enumerate(matrix_order):
    ax3.add_patch(Rectangle((i, 0), 1, 1, linewidth=0, edgecolor='black', facecolor=species_color[indiv]))
    axD.add_patch(Rectangle((0, i), 1, 1, linewidth=0, edgecolor='black', facecolor=species_color[indiv]))
#ax3.text(-30, 0.5, 'Species', fontsize=10, verticalalignment='center', horizontalalignment='right')  

# extra axes for heatmap color bar to allow x sharing of the other three axes
cbar_ax = fig.add_axes([.93, .3, .02, .3])
# plot heatmap in bottom panel
colors = ['white', 'mediumblue', 'darkviolet', 'orangered',  'darkorange'] 
cmap = LinearSegmentedColormap.from_list('custom colormap', colors, N=256)
sns.heatmap(ordered_matrix.to_numpy(), cmap=cmap, ax=ax4, cbar_ax=cbar_ax)

# add squares with labels along diagonal (maybe to label indentified clusters)
fontsize=11
for label, start, size in [('A', 0, 25), ('B', 25, 26), ('C', 51, 12), ('D', 63, 67),
                          ('E', 130, 27), ('F', 157, 10), ('G', 167, 4), ('H', 171, 52)]:
    ax4.add_patch(Rectangle((start, ordered_matrix.shape[0] - start), size, - size, 
                            linewidth=0.4, edgecolor='black', facecolor='none'))
    ax4.text(start, ordered_matrix.shape[0] - start-size, label, fontsize=fontsize,
                    verticalalignment='bottom', 
                    horizontalalignment='left')    

# come additional squares (maybe to highlight particular admixture signals)
label, x, y, width, height = 'X', 63, 157, 67, 10
ax4.add_patch(Rectangle((x, ordered_matrix.shape[0] - y), width, -height, 
                        linewidth=0.4, edgecolor='black', facecolor='none'))
ax4.text(x, ordered_matrix.shape[0] - y-height, label, fontsize=fontsize,
                verticalalignment='bottom', 
                horizontalalignment='left')    
    
# remove axes from all plots
ax1.axis('off')
ax2.axis('off') 
ax3.axis('off') 
ax4.axis('off') 
axD.axis('off') 

# lines around the matrix
ax4.axhline(y=0, color='black',linewidth=1)
ax4.axhline(y=ordered_matrix.shape[1], color='black',linewidth=1)
ax4.axvline(x=0, color='black',linewidth=1)
ax4.axvline(x=ordered_matrix.shape[0], color='black',linewidth=1)

# lines around the colorbar
cbar_ax.axhline(y=0, color='black',linewidth=1)
cbar_ax.tick_params(right=False)

In [ ]:
# three panels
fig, ((axA, ax1), (axB, ax2), (axC, ax3))  = plt.subplots(3, 2, sharex='col', sharey='row',
                        gridspec_kw={'height_ratios':[10,1,40], 
                                     'width_ratios':[1,40],
                                     'hspace':0.01,
                                     'wspace':0.01}, 
                        figsize=(8, 9))

# remove axes we do not need
axA.remove()
axB.remove()

# plot the tree in top panel
leaf_info = plot_tree(Tree(tree), ax1, flip=True, 
                      axis=False, linewidth=0.6,
                      markers=True, 
                      markersize=1,
                      labels=False,
                      margins=(0, 0, 0, 0), 
                      fontsize=4)

# add colored rectangles or each individual to the middle
for i, indiv in enumerate(matrix_order):
    ax2.add_patch(Rectangle((i, 0), 1, 1, linewidth=0, edgecolor='black', facecolor=species_colors[indiv]))
    axC.add_patch(Rectangle((0, i), 1, 1, linewidth=0, edgecolor='black', facecolor=species_colors[indiv]))

# extra axes for heatmap color bar to allow x sharing of the other three axes
cbar_ax = fig.add_axes([.93, .3, .02, .3])
# plot heatmap in bottom panel
colors = ['white', 'mediumblue', 'darkviolet', 'orangered',  'darkorange'] 
cmap = LinearSegmentedColormap.from_list('custom colormap', colors, N=256)
sns.heatmap(ordered_matrix.to_numpy(), cmap=cmap, ax=ax3, cbar_ax=cbar_ax)

# add squares with labels along diagonal (maybe to label indentified clusters)
fontsize=11
for label, start, size in [('A', 50, 30), ('B', 100, 50), ('C', 150, 30)]:
    ax3.add_patch(Rectangle((start, ordered_matrix.shape[0] - start), size, - size, 
                            linewidth=0.4, edgecolor='black', facecolor='none'))
    ax3.text(start, ordered_matrix.shape[0] - start-size, label, fontsize=fontsize,
                    verticalalignment='bottom', 
                    horizontalalignment='left')    

# come additional squares (maybe to highlight particular admixture signals)
label, x, y, width, height = 'K', 20, 120, 50, 20
ax3.add_patch(Rectangle((x, ordered_matrix.shape[0] - y), width, -height, 
                        linewidth=0.4, edgecolor='black', facecolor='none'))
ax3.text(x, ordered_matrix.shape[0] - y-height, label, fontsize=fontsize,
                verticalalignment='bottom', 
                horizontalalignment='left')    
    
# remove axes from all plots
ax1.axis('off')
ax2.axis('off') 
ax3.axis('off') 
axC.axis('off') 

# lines around the matrix
ax3.axhline(y=0, color='black',linewidth=1)
ax3.axhline(y=ordered_matrix.shape[1], color='black',linewidth=1)
ax3.axvline(x=0, color='black',linewidth=1)
ax3.axvline(x=ordered_matrix.shape[0], color='black',linewidth=1)

# lines around the colorbar
cbar_ax.axhline(y=0, color='black',linewidth=1)
cbar_ax.tick_params(right=False)